# Row level security

In this demo we will use three users

- David and George: just business users with limited access to a table
- Dev: some kind of "superuser" who should see all data in the same table

First step is a preparation of users.

In [ ]:
use Demo
GO

if exists(select * from sys.server_principals where name = 'George')
    drop login George

create login George with password = 'Pa$$w0rd', check_policy = off, default_database = master, default_language = English
go

if exists(select * from sys.database_principals where name = 'George')
    drop user George;
go

create user George for login George with default_schema = dbo
go

select name, principal_id from sys.database_principals

grant select on schema::Sales to George
GO

-- Dev user already has all permissions needed because he is a member of the db_owner role

Let's adjust the Sales.SalesCases table. Then we will add a couple of records.

In [ ]:
use Demo
GO
alter table Sales.SalesCases add created_by nvarchar(20) not NULL
go

insert Sales.SalesCases (SomeValue, created_by) values (4, 'David')
insert Sales.SalesCases (SomeValue, created_by) values (6, 'George')
go 5

select * from Sales.SalesCases

## Creating security objects

### Security predicate
A in-line table-valued function with one (and strictly one) parameter. The parameter serves as a binding value for filtering data from table.

In [ ]:
use Demo
GO

create or alter function Sales.fnSecurityPredicate(@user nvarchar(20))
returns table
with SCHEMABINDING
as
return select 1 as true_result where USER_NAME() = @user
go

### Security policy
An object which binds the security predicate with filtered table. The security policy could be enabled or disabled.

In [ ]:
use Demo
go

create security policy Sales.SalesCasesPolicy
add filter predicate Sales.fnSecurityPredicate(created_by)
on Sales.SalesCases
with (state = on)
go


### Testing the policy
We'll execute the same query for all three users - David, George and Dev

In [ ]:
use Demo
go

execute as user = 'David'
select * from Sales.SalesCases
revert

execute as user = 'George'
select * from Sales.SalesCases
revert

execute as user = 'Dev'
select * from Sales.SalesCases
revert

**Oh! Wait! Dev user is a "superuser", isn't it?**

Let's adjust the security predicate for him. Only way is to add an exceptional predicate to the WHERE clause of the function.

In [ ]:
use Demo
GO

drop security policy Sales.SalesCasesPolicy
go

create or alter function Sales.fnSecurityPredicate(@user nvarchar(20))
returns table
with SCHEMABINDING
as
return select 1 as true_result where USER_NAME() = @user or USER_NAME() = 'Dev'
go

create security policy Sales.SalesCasesPolicy
add filter predicate Sales.fnSecurityPredicate(created_by)
on Sales.SalesCases
with (state = on)
go